DATASET liiklusonnetused

In [1]:
import pandas as pd
import glob
import sqlite3
import csv
import pyproj

In [2]:
df_original = pd.read_csv('liiklusonnetused_until_2021.csv')
print(df_original.columns)

Index(['Juhtumi nr', 'Toimumisaeg', 'Isikuid', 'Hukkunuid', 'Vigastatuid',
       'Sõidukeid', 'Aadress (PPA)', 'Maja nr (PPA)', 'Tänav (PPA)',
       'Ristuv tänav (PPA)', 'Maakond (PPA)', 'Omavalitsus (PPA)',
       'Asustus (PPA)', 'Asula', 'Liiklusõnnetuse liik [1]',
       'Liiklusõnnetuse liik [3]', 'Kergliikurijuhi osalusel',
       'Jalakäija osalusel', 'Kaassõitja osalusel',
       'Maastikusõiduki juhi osalusel',
       'Eaka (65+) mootorsõidukijuhi osalusel', 'Bussijuhi osalusel',
       'Veoautojuhi osalusel', 'Ühissõidukijuhi osalusel',
       'Sõiduautojuhi osalusel', 'Mootorratturi osalusel',
       'Mopeedijuhi osalusel', 'Jalgratturi osalusel', 'Alaealise osalusel',
       'Turvavarustust mitte kasutanud isiku osalusel',
       'Esmase juhiloa omaniku osalusel', 'Mootorsõidukijuhi osalusel',
       'Tüüpskeemi nr', 'Tüüpskeem [2]', 'Tee tüüp [1]', 'Tee tüüp [2]',
       'Tee element [1]', 'Tee element [2]', 'Tee objekt [2]', 'Kurvilisus',
       'Tee tasasus', 'Tee sei

In [3]:
#Keeping only neccesary columns
columns_to_keep = ['Juhtumi nr', 'Toimumisaeg', 'Asula', 'Liiklusõnnetuse liik [3]','GPS X', 'GPS Y']
df_clean = df_original[columns_to_keep]
print(df_clean.head())

      Juhtumi nr       Toimumisaeg Asula          Liiklusõnnetuse liik [3]  \
0  2000140000057  24.10.2014 08:45   JAH            Kokkupõrge jalakäijaga   
1  2000140000067  24.10.2014 13:45   JAH  Kokkupõrge ees liikuva sõidukiga   
2  2000140000123           8/11/14   JAH  Kokkupõrge ees liikuva sõidukiga   
3  2000140000235  17.11.2014 17:32    EI  Kokkupõrge vastutuleva sõidukiga   
4  2000150000442  28.04.2015 07:55   JAH      Kokkupõrge sõidukiga küljelt   

       GPS X     GPS Y  
0  6588678.0  542647.0  
1  6589522.0  541467.0  
2  6593961.0  547646.0  
3  6569324.0  516628.0  
4  6586430.0  541953.0  


In [4]:
#Delete all rows that does not have time
num_rows = df_original.shape[0]
df_clean = df_clean[~df_clean['Toimumisaeg'].str.contains(r'\d+/\d+/\d{2}')]
num_rows_new = df_clean.shape[0]
#print(df_clean.head())
#print(num_rows) 
#print(num_rows_new) #15702 rows originally and now 9322, its almost half the data :(

# Convert "Toimumisaeg" column to datetime format
df_clean['Toimumisaeg'] = pd.to_datetime(df_clean['Toimumisaeg'], dayfirst=True)
#print(df_clean.head())

# Extract day, month, year, hour, and minute into separate columns
df_clean.loc[:, 'Toimumisaeg'] = pd.to_datetime(df_clean['Toimumisaeg'], dayfirst=True)
df_clean.loc[:, 'Day'] = df_clean['Toimumisaeg'].dt.day
df_clean.loc[:, 'Month'] = df_clean['Toimumisaeg'].dt.month
df_clean.loc[:, 'Year'] = df_clean['Toimumisaeg'].dt.year
df_clean.loc[:, 'Hour'] = df_clean['Toimumisaeg'].dt.hour
df_clean.loc[:, 'Minute'] = df_clean['Toimumisaeg'].dt.minute


print(df_clean.head())

      Juhtumi nr         Toimumisaeg Asula          Liiklusõnnetuse liik [3]  \
0  2000140000057 2014-10-24 08:45:00   JAH            Kokkupõrge jalakäijaga   
1  2000140000067 2014-10-24 13:45:00   JAH  Kokkupõrge ees liikuva sõidukiga   
3  2000140000235 2014-11-17 17:32:00    EI  Kokkupõrge vastutuleva sõidukiga   
4  2000150000442 2015-04-28 07:55:00   JAH      Kokkupõrge sõidukiga küljelt   
6  2210140001932 2014-10-24 14:00:00    EI  Kokkupõrge vastutuleva sõidukiga   

       GPS X     GPS Y  Day  Month  Year  Hour  Minute  
0  6588678.0  542647.0   24     10  2014     8      45  
1  6589522.0  541467.0   24     10  2014    13      45  
3  6569324.0  516628.0   17     11  2014    17      32  
4  6586430.0  541953.0   28      4  2015     7      55  
6  6575538.0  529034.0   24     10  2014    14       0  


In [5]:
#CHANGING COLUMN NAMES

# Create a dictionary to map the old column names to the new column names
column_mapping = {
    'Juhtumi nr': 'Juhtumi_nr',
    'Liiklusõnnetuse liik [3]': 'Liiklusonnetuse_liik',
    'GPS X': 'GPS_X',
    'GPS Y': 'GPS_Y',
}

# Use the 'rename()' method to change the column names
df_clean = df_clean.rename(columns=column_mapping)

# Print the modified DataFrame with the new column names
print(df_clean.head())

      Juhtumi_nr         Toimumisaeg Asula              Liiklusonnetuse_liik  \
0  2000140000057 2014-10-24 08:45:00   JAH            Kokkupõrge jalakäijaga   
1  2000140000067 2014-10-24 13:45:00   JAH  Kokkupõrge ees liikuva sõidukiga   
3  2000140000235 2014-11-17 17:32:00    EI  Kokkupõrge vastutuleva sõidukiga   
4  2000150000442 2015-04-28 07:55:00   JAH      Kokkupõrge sõidukiga küljelt   
6  2210140001932 2014-10-24 14:00:00    EI  Kokkupõrge vastutuleva sõidukiga   

       GPS_X     GPS_Y  Day  Month  Year  Hour  Minute  
0  6588678.0  542647.0   24     10  2014     8      45  
1  6589522.0  541467.0   24     10  2014    13      45  
3  6569324.0  516628.0   17     11  2014    17      32  
4  6586430.0  541953.0   28      4  2015     7      55  
6  6575538.0  529034.0   24     10  2014    14       0  


In [6]:
#MISSING VALUES
# Check for missing values in the DataFrame
missing_values = df_clean.isnull()

missing_counts = df_clean.isnull().sum()
print(missing_counts) #1273-l real ei ole GPS koordinaate

Juhtumi_nr                 0
Toimumisaeg                0
Asula                      0
Liiklusonnetuse_liik       0
GPS_X                   1273
GPS_Y                   1277
Day                        0
Month                      0
Year                       0
Hour                       0
Minute                     0
dtype: int64


In [7]:
#DELETING ROWS WITH MISSING GPS COORDINATES
df_clean.dropna(subset=['GPS_X'], inplace=True)
df_clean.dropna(subset=['GPS_Y'], inplace=True)
print(df_clean.shape)  #Alles on 8049 rida


(8045, 11)


In [8]:
#LIIKLUSONNETUSE LIIK VALUES

# Get the unique values and their counts in the 'Liiklusonnetuse_liik' column
value_counts = df_clean['Liiklusonnetuse_liik'].value_counts()

# Print the unique values and their counts
print(value_counts)

Liiklusonnetuse_liik
Kokkupõrge jalakäijaga               1846
Kokkupõrge sõidukiga küljelt         1597
Sõiduki teelt väljasõit              1132
Kokkupõrge vastutuleva sõidukiga      590
Kokkupõrge ees liikuva sõidukiga      564
Sõiduki ümberpaiskumine teel          544
Kokkupõrge teevälise takistusega      491
Muu liiklusõnnetus                    364
Kokkupõrge ees seisva sõidukiga       311
Kukkumine ühissõidukis                250
Sõidukite külgkokkupõrge              147
Kokkupõrge loomaga                    101
Kokkupõrge teel oleva takistusega      67
Teadmata                               41
Name: count, dtype: int64


In [9]:
#CONVERT COORDINATES TO LATITUDE AND LONGITUDE


# Create a transformer
transformer = pyproj.Transformer.from_crs("EPSG:3301", "EPSG:4326")

# Convert coordinates to latitude and longitude
df_clean[['Latitude', 'Longitude']] = df_clean.apply(lambda row: pd.Series(transformer.transform(row['GPS_X'], row['GPS_Y'])), axis=1)



In [10]:
print(df_clean.head())

      Juhtumi_nr         Toimumisaeg Asula              Liiklusonnetuse_liik  \
0  2000140000057 2014-10-24 08:45:00   JAH            Kokkupõrge jalakäijaga   
1  2000140000067 2014-10-24 13:45:00   JAH  Kokkupõrge ees liikuva sõidukiga   
3  2000140000235 2014-11-17 17:32:00    EI  Kokkupõrge vastutuleva sõidukiga   
4  2000150000442 2015-04-28 07:55:00   JAH      Kokkupõrge sõidukiga küljelt   
6  2210140001932 2014-10-24 14:00:00    EI  Kokkupõrge vastutuleva sõidukiga   

       GPS_X     GPS_Y  Day  Month  Year  Hour  Minute   Latitude  Longitude  
0  6588678.0  542647.0   24     10  2014     8      45  59.433796  24.751479  
1  6589522.0  541467.0   24     10  2014    13      45  59.441488  24.730850  
3  6569324.0  516628.0   17     11  2014    17      32  59.261884  24.291529  
4  6586430.0  541953.0   28      4  2015     7      55  59.413688  24.738815  
6  6575538.0  529034.0   24     10  2014    14       0  59.316999  24.509857  


In [11]:

#CREATING a table INTO DATABASE

# Connect to the SQLite database
conn = sqlite3.connect('database.db')

# Create a cursor object to execute SQL statements
cursor = conn.cursor()

# Define the CREATE TABLE statement
create_table_query = '''
CREATE TABLE IF NOT EXISTS liiklusonnetus (
Juhtumi_nr INTEGER,
Toimumisaeg TEXT,
Asula TEXT,
Liiklusonnetuse_liik TEXT,
GPS_X REAL,
GPS_Y REAL,
Day INTEGER,
Month INTEGER,
Year INTEGER,
Hour INTEGER,
Minute INTEGER,
Longitude REAL,
Latitude REAL
);
'''
cursor.execute(create_table_query)

conn.commit()
cursor.close()
conn.close()

In [12]:
# Connect to the SQLite database
conn = sqlite3.connect('database.db')

# Write dataframe to the table in the database
df_clean.to_sql('liiklusonnetus', conn, if_exists='append', index=False)

# Close the connection
conn.close()

DATASET liiklusloendus

In [14]:
#CHANGE COLUMN NAMES IN ALL CSV FILES in A FOLDER
# Create a dictionary to map the old column names to the new column names
column_mapping = {
    '1': 'Motorcycle',
    '2': 'Car',
    '3': 'Car+Trailer',
    '4': 'Heavy_Van',
    '5': 'Light_Goods',
    '6': 'Truck',
    '7': 'Truck+Trailer',
    '8': 'Articulated_Vehicle',
    '9': 'Minibus',
    '10': 'Bus',
}

folder_path = 'liiklusloendus'

file_paths = glob.glob(folder_path + '/*.csv')  # Get all CSV file paths in the folder

for file in file_paths:
    df_liiklus = pd.read_csv(file)
    df_liiklus.rename(columns=column_mapping, inplace=True)
    df_liiklus.to_csv(file, index=False)


C:\Users\jarve\AppData\Local\Temp\ipykernel_5172\1526013909.py:21: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_liiklus = pd.read_csv(file)
C:\Users\jarve\AppData\Local\Temp\ipykernel_5172\1526013909.py:21: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_liiklus = pd.read_csv(file)
C:\Users\jarve\AppData\Local\Temp\ipykernel_5172\1526013909.py:21: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_liiklus = pd.read_csv(file)


In [15]:
#CREATING A TABLE INTO DATABASE

# Connect to the SQLite database
conn = sqlite3.connect('database.db')

# Create a cursor object to execute SQL statements
cursor = conn.cursor()

# Define the CREATE TABLE statement
create_table_query = '''
CREATE TABLE IF NOT EXISTS liiklusloendus (
id INTEGER PRIMARY KEY AUTOINCREMENT,
Motorcycle INTEGER,
Car INTEGER,
Car_Plus_Trailer INTEGER,
Heavy_Van INTEGER,
Light_Goods INTEGER,
Truck INTEGER,
Truck_Plus_Trailer INTEGER,
Articulated_Vehicle INTEGER,
Minibus INTEGER,
Bus INTEGER,
liiklusloend_id TEXT,
Kanal INTEGER,
Aeg TEXT,
"less40" INTEGER,
"40to50" INTEGER,
"50to60" INTEGER,
"60to70" INTEGER,
"70to80" INTEGER,
"80to90" INTEGER,
"90to100" INTEGER,
"100to110" INTEGER,
"110to120" INTEGER,
"120to130" INTEGER,
"more130" INTEGER
);
'''
cursor.execute(create_table_query)

conn.commit()
cursor.close()
conn.close()

In [16]:
#WRITING ALL VALUES FROM CSV FILES INTO DATABASE

conn = sqlite3.connect('database.db')
cursor = conn.cursor()

folder_path = 'liiklusloendus'

file_paths = glob.glob(folder_path + '/*.csv')  # Get all CSV file paths in the folder

for file in file_paths:
    with open(file, 'r') as fail:
        csv_reader = csv.reader(fail)
        next(csv_reader) #skip the header row

        for row in csv_reader:
            cursor.execute("INSERT INTO liiklusloendus VALUES (NULL,?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", row)

conn.commit()
conn.close()

DATASET liiklusloendusseadmed

In [17]:
df_seadmed = pd.read_csv('liiklusloendusseadmed.csv')
print(df_seadmed.columns)

missing_values = df_seadmed.isnull()

missing_counts = df_seadmed.isnull().sum()
print(missing_counts)

print(df_seadmed.head())

Index(['Name', 'Connection ID', 'Road nr', 'Road km', 'County', 'Lon', 'Lat'], dtype='object')
Name             0
Connection ID    0
Road nr          0
Road km          0
County           0
Lon              0
Lat              0
dtype: int64
                       Name Connection ID  Road nr  Road km         County  \
0           LOO 1_13236 VBV         0cfcf        1    13.20       Harjumaa   
1  PRÜGILA RIST 1_17794 VBV         25785        1    17.79       Harjumaa   
2       KODASOO 1_32100 VBV         077a6        1    32.10       Harjumaa   
3        VIITNA 1_73241 VBV         67241        1    73.20  Lääne-Virumaa   
4         SÄMI 1_109455 VBV         387ad        1   109.40  Lääne-Virumaa   

         Lon       Lat  
0  24.964570  59.44959  
1  25.044434  59.45134  
2  25.290380  59.44065  
3  26.008360  59.46308  
4  26.582640  59.37418  


In [18]:
#CREATING A TABLE INTO DATABASE

# Connect to the SQLite database
conn = sqlite3.connect('database.db')

# Create a cursor object to execute SQL statements
cursor = conn.cursor()

# Define the CREATE TABLE statement
create_table_query = '''
CREATE TABLE IF NOT EXISTS seadmed (
Name TEXT PRIMARY KEY,
'Connection ID' TEXT,
'Road nr' INTEGER,
'Road km' REAL,
'County' TEXT,
Lon REAL,
Lat REAL
);
'''
cursor.execute(create_table_query)

conn.commit()
cursor.close()
conn.close()

In [19]:
# Connect to the SQLite database
conn = sqlite3.connect('database.db')

# Write dataframe to the table in the database
df_seadmed.to_sql('seadmed', conn, if_exists='append', index=False)

# Close the connection
conn.close()